In [1]:
! mpicxx --version

g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
! dpkg -l | grep libopencv

ii  libopencv-calib3d-dev:amd64               4.5.4+dfsg-9ubuntu4+jammy0              amd64        development files for libopencv-calib3d4.5d
ii  libopencv-calib3d4.5d:amd64               4.5.4+dfsg-9ubuntu4+jammy0              amd64        computer vision Camera Calibration library
ii  libopencv-contrib-dev:amd64               4.5.4+dfsg-9ubuntu4+jammy0              amd64        development files for libopencv-contrib4.5d
ii  libopencv-contrib4.5d:amd64               4.5.4+dfsg-9ubuntu4+jammy0              amd64        computer vision contrlib library
ii  libopencv-core-dev:amd64                  4.5.4+dfsg-9ubuntu4+jammy0              amd64        development files for libopencv-core4.5d
ii  libopencv-core4.5d:amd64                  4.5.4+dfsg-9ubuntu4+jammy0              amd64        computer vision core library
ii  libopencv-dev                             4.5.4+dfsg-9ubuntu4+jammy0              amd64        development files for opencv
ii  libopencv-dnn-dev:amd64                 

In [16]:
!apt-get install -qq gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5
!ln -s /usr/bin/g++-5

!sudo apt-get update
!sudo apt-get upgrade

#Install Dependencies
!sudo apt-get install -y build-essential
!sudo apt-get install -y cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev
#The following command is needed to process images:
!sudo apt-get install -y python-dev python-numpy libtbb2 libtbb-dev libjpeg-dev libpng-dev libtiff-dev libjasper-dev libdc1394-22-dev
#To process videos:
!sudo apt-get install -y libavcodec-dev libavformat-dev libswscale-dev libv4l-dev
!sudo apt-get install -y libxvidcore-dev libx264-dev
#For GUI:
!sudo apt-get install -y libgtk-3-dev
#For optimization:
!sudo apt-get install -y libatlas-base-dev gfortran pylint
!wget https://github.com/opencv/opencv/archive/3.4.0.zip -O opencv-3.4.0.zip
!sudo apt-get install unzip
!unzip opencv-3.4.0.zip
%cd opencv-3.4.0
!mkdir build
%cd build
!cmake -D WITH_TBB=ON -D WITH_OPENMP=ON -D WITH_IPP=ON -D CMAKE_BUILD_TYPE=RELEASE -D BUILD_EXAMPLES=OFF -D WITH_NVCUVID=ON -D WITH_CUDA=OFF -D BUILD_DOCS=OFF -D BUILD_PERF_TESTS=OFF -D BUILD_TESTS=OFF -D WITH_CSTRIPES=ON -D WITH_OPENCL=ON CMAKE_INSTALL_PREFIX=/usr/local/ ..
!make -j`nproc`
!sudo make install


Streaming output truncated to the last 5000 lines.
  inflating: opencv-3.4.0/platforms/ios/Info.Dynamic.plist.in  
  inflating: opencv-3.4.0/platforms/ios/Info.plist.in  
  inflating: opencv-3.4.0/platforms/ios/build_framework.py  
   creating: opencv-3.4.0/platforms/ios/cmake/
   creating: opencv-3.4.0/platforms/ios/cmake/Modules/
   creating: opencv-3.4.0/platforms/ios/cmake/Modules/Platform/
  inflating: opencv-3.4.0/platforms/ios/cmake/Modules/Platform/iOS.cmake  
   creating: opencv-3.4.0/platforms/ios/cmake/Toolchains/
  inflating: opencv-3.4.0/platforms/ios/cmake/Toolchains/Toolchain-iPhoneOS_Xcode.cmake  
  inflating: opencv-3.4.0/platforms/ios/cmake/Toolchains/Toolchain-iPhoneSimulator_Xcode.cmake  
  inflating: opencv-3.4.0/platforms/ios/cmake/Toolchains/common-ios-toolchain.cmake  
  inflating: opencv-3.4.0/platforms/ios/readme.txt  
   creating: opencv-3.4.0/platforms/js/
  inflating: opencv-3.4.0/platforms/js/README.md  
  inflating: opencv-3.4.0/platforms/js/build_js.py  

In [19]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.27)
project(HPC_Project)

set(CMAKE_CXX_STANDARD 20)

find_package(MPI REQUIRED)
find_package(OpenCV REQUIRED)

add_executable(HPC_Project main.cpp)


target_link_libraries(HPC_Project PRIVATE MPI::MPI_CXX ${MPI_LIBRARIES} ${OpenCV_LIBS})
#OpenCV::core OpenCV::imgproc OpenCV::highgui

Writing CMakeLists.txt


In [20]:
%%writefile main.cpp

#include <iostream>
#include <opencv2/opencv.hpp>
#include <opencv2/highgui.hpp>
#include <opencv2/imgproc.hpp>
#include <mpi.h>

#include <chrono>
#include <vector>

int main()
{
  MPI_Init(NULL, NULL); // Initialised MPI
  // Find my (processors') Unique ID - (also called as rank)
  int my_rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  // Find the total number of processors
  int world_size;
  MPI_Comm_size(MPI_COMM_WORLD, &world_size);
  MPI_Barrier(MPI_COMM_WORLD);
  std::cout << "My rank is " << my_rank << " among " << world_size << " Processors.." << std::endl;
  MPI_Finalize(); // MPI Terminated
  return 0;
}

Writing main.cpp


In [22]:
! make clean && cmake -B build -DCMAKE_BUILD_TYPE=Debug && make

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found MPI_C: /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so (found version "3.1") 
-- Found MPI_CXX: /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi_cxx.so (found version "3.1") 
-- Found MPI: TRUE (found version "3.1")  
-- Found OpenCV: /usr (found version "4.5.4") 
-- Configuring done (2.8s)
-- Generating done (0.0s)
-- Build files have been written to: /content/opencv-3.4.0/build/build
[  0%] Generate opencv.pc
CMake Deprecation Warning at /content/opencv-3.4.0/cmake/OpenCVGenPkgconfig

In [23]:
!mpicxx main.cpp -o test

main.cpp:3:10: fatal error: opencv2/opencv.hpp: No such file or directory
    3 | #include <opencv2/opencv.hpp>
      |          ^~~~~~~~~~~~~~~~~~~~
compilation terminated.


In [15]:
!mpirun -n 8 --allow-run-as-root -oversubscribe --use-hwthread-cpus ./test

My rank is 2 among 8 Processors..
My rank is 3 among 8 Processors..
My rank is 6 among 8 Processors..
My rank is 0 among 8 Processors..
My rank is 4 among 8 Processors..
My rank is 1 among 8 Processors..
My rank is 5 among 8 Processors..
My rank is 7 among 8 Processors..
